# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from my_keys3 import g2_key

In [2]:
# pip install matplotlib==3.2

In [3]:
# pip install requests

In [4]:
# pip install os

In [5]:
# pip install matplotlib

In [6]:
# pip install requests

In [7]:
# pip install gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_df = pd.read_csv('./clean_weather_info.csv')
weather_df.head(10)

,City,Latitude,Longtitude,Max Temperature,Humidity,Clouds,Wind Speed,Country,Weather Date
0,Port Alfred,-33.5906,26.8910,73.22,88,97,8.81,ZA,1643226535
1,Mataura,-46.1927,168.8643,52.48,96,61,4.63,NZ,1643226535
2,Blagoyevo,63.3667,47.9167,13.41,97,100,3.58,RU,1643226536
3,Thompson,55.7435,-97.8558,6.96,72,75,12.66,CA,1643226334
4,Dikson,73.5069,80.5464,13.68,92,100,28.97,RU,1643226536
5,Beringovskiy,63.0500,179.3167,-6.65,99,100,3.38,RU,1643226537
6,Zyryanka,65.7500,150.8500,-39.89,99,60,6.85,RU,1643226537
7,Albany,42.6001,-73.9662,21.52,44,14,8.01,US,1643226517
8,Ilulissat,69.2167,-51.1000,17.62,73,75,16.11,GL,1643226538
9,Castro,-24.7911,-50.0119,86.41,51,27,5.64,BR,1643226457


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g2_key)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(
    weather_df[['Latitude', 'Longtitude']], weights=weather_df['Humidity'], max_intensity=30, point_radius=2.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#A max temperature lower than 80 degrees but higher than 70.
temperature_filtered_df = weather_df[(weather_df['Max Temperature'] <= 80) & (weather_df['Max Temperature'] > 70)]

# Wind speed less than 10 mph.
speed_filtered_df = temperature_filtered_df[temperature_filtered_df['Wind Speed'] < 10]

# Zero cloudiness.
cloud_f_df = speed_filtered_df[speed_filtered_df['Clouds'] == 0]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
import json

hotel_df = cloud_f_df.copy()

hotel_names = []
for _, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longtitude = row['Longtitude']    

    # Query places api for nearest hotel, if exists add to list
    endpoint_url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=hotel&location={latitude},{longtitude}&radius=5000&region=us&key={g2_key}"
    response = requests.get(endpoint_url, params = {})
    results = json.loads(response.content)['results']
    
    hotel_name = None
    if len(results) > 0:
        hotel_name = results[0]['name']

    hotel_names.append(hotel_name)

hotel_df['Hotel Name'] = hotel_names
hotel_df['Lat'] = hotel_df['Latitude']
hotel_df['Lng'] = hotel_df['Longtitude']
hotel_df.head(10)

,City,Latitude,Longtitude,Max Temperature,Humidity,Clouds,Wind Speed,Country,Weather Date,Hotel Name,Lat,Lng
56,Cape Town,-33.9258,18.4232,70.21,78,0,4.00,ZA,1643226556,None,-33.9258,18.4232
103,Tecámac,19.7333,-98.9667,70.99,25,0,4.25,MX,1643226574,None,19.7333,-98.9667
381,Makakilo City,21.3469,-158.0858,77.29,70,0,5.75,US,1643226683,None,21.3469,-158.0858
535,Kétou,7.3633,2.5998,78.21,31,0,5.66,BJ,1643226742,None,7.3633,2.5998


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(
            locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))